In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import logging
from itertools import count
from pathlib import Path
from uBERTa.datasets_generator import DatasetGenerator

In [3]:
# Optionally, turn on logging to see the generation progress
logging.basicConfig(level=logging.INFO)

In [4]:
# Replace with relevant download paths
DS_PATH, REF_PATH = Path('../../data/DS_BASE.tsv'), Path('../../data/initial/hg38.fa')

In [5]:
?DatasetGenerator

Init signature:
DatasetGenerator(
    ds_path: pathlib.Path,
    ref_path: pathlib.Path,
    neg_multiplier: int,
    neg_fractions: Tuple[float, float, float, float],
    pos_fractions: Tuple[float, float, float] = (1.0, 1.0, 1.0),
    level_ts: float = 0,
    flank_size: int = 100,
    kmer_size: Optional[int] = None,
    use_analyzed: bool = True,
    col_names: uBERTa.base.ColNames = ColNames(chrom='Chrom', start='StartCodonStart', end='StartCodonEnd', codon='StartCodonFetched', strand='Strand', group='Group', level='LevelStartCodonStartFetchedAround2', analyzed='IsAnalyzed', positive='IsPositive'),
    drop_meta: bool = False,
)
Docstring:     
An interface class to generate (u)ORF datasets.

Class is a container for the generation settings.
Use case: generate datasets with different examples, but preserving the composition,
i.e., the balance between different kinds of positive and negative examples.

    1. First, it samples different groups of positive uORFs (u, ma, m).
    2. T

- Most of the sampling time is spent on scanning the genome: triplet-wise search for a particular start-codon, starting from a random location.
- The number of __expected__ negative samples may be slightly higher than the number of __actually__ sampled ones due to rounding while calculating the number of samples per start-codon.

In [16]:
# Set up the generator
dsg = DatasetGenerator(
    DS_PATH, REF_PATH, 
    neg_multiplier=1, 
    neg_fractions=(
        0.0,  # Completely random samples 
        0.1,  # Random samples centered on start codons
        0.45,  # Valid uORFs without experimental support
        0.45   # Valid uORFs with experimental support
    ),
    pos_fractions=(
        0.8,  # u samples  (uORF start codons)
        0.8,  # ma samples (alternative start codons)
        0.5   # m samples  (CDS start codons)
    ),
    flank_size=50
)

# Infinite ds generator with the given params
gen = (dsg() for _ in count())

In [17]:
%%time
next(gen).head()

INFO:uBERTa.datasets_generator:Pre-filtering initial dataset with 1147700 records
INFO:uBERTa.datasets_generator:Filtered to 253935 analyzed records
INFO:uBERTa.datasets_generator:Obtained a dataset with 253935 records
INFO:uBERTa.datasets_generator:Found 7150 positive and 246785 (potential) negative records
INFO:uBERTa.datasets_generator:Will sample 3583=4479 * 0.8 positive examples of group u
INFO:uBERTa.datasets_generator:Will sample 371=464 * 0.8 positive examples of group ma
INFO:uBERTa.datasets_generator:Will sample 1103=2207 * 0.5 positive examples of group m
INFO:uBERTa.datasets_generator:Sampled 5057 positive examples
INFO:uBERTa.datasets_generator:Expecting around 5057 total negative examples
INFO:uBERTa.datasets_generator:Num samples after the per-codon correction: 5040
INFO:uBERTa.datasets_generator:Found 218 codons ATG
INFO:uBERTa.datasets_generator:Found 141 codons CTG
INFO:uBERTa.datasets_generator:Found 54 codons GTG
INFO:uBERTa.datasets_generator:Found 27 codons TTG
IN

CPU times: user 10.2 s, sys: 1.54 s, total: 11.7 s
Wall time: 1min 44s


,Chrom,StartCodonStart,Strand,IsPositive,Seq
0,chr19,50384364,+,1,GGCTCTGGGCTTGCGCGCGCGGGAGTCAGGGGTCACGGCGGCGTAG...
1,chr1,108200176,-,1,GCCCGGCCTCGCGCTGGTCCCGGTCTCGCCCCGCAGCCCTCGATCT...
2,chr6,166956438,-,1,CTGTGGGTCGCGAAGCAGAGCCCGGGACGTGCGCGCTTGGTGCACG...
3,chrX,63755027,-,1,GTGTCTGGCAGCCGGGAGCCCAGCGAAGAGAGCGAGCAAGCTTAGG...
4,chr12,56042015,+,1,TGGATAAATAAACACTAGGAACGCATTTCCACCCTAGATTTCAGCA...


In [7]:
# Drop meta columns, leave only sequence and class
dsg.drop_meta = True
dsg().head()

INFO:uBERTa.datasets_generator:Obtained a dataset with 830191 records
INFO:uBERTa.datasets_generator:Found 7426 positive and 822765 negative records
INFO:uBERTa.datasets_generator:Will sample 3680=4600 * 0.8 positive examples of group u
INFO:uBERTa.datasets_generator:Will sample 388=485 * 0.8 positive examples of group ma
INFO:uBERTa.datasets_generator:Will sample 1170=2341 * 0.5 positive examples of group m
INFO:uBERTa.datasets_generator:Sampled 5238 positive examples
INFO:uBERTa.datasets_generator:Expecting around 5238 total negative examples
INFO:uBERTa.datasets_generator:Num samples after the per-codon correction: 5234
INFO:uBERTa.datasets_generator:Found 1158 codons ATG
INFO:uBERTa.datasets_generator:Found 736 codons CTG
INFO:uBERTa.datasets_generator:Found 281 codons GTG
INFO:uBERTa.datasets_generator:Found 146 codons ACG
INFO:uBERTa.datasets_generator:Found 141 codons TTG
INFO:uBERTa.datasets_generator:Found 73 codons ATC
INFO:uBERTa.datasets_generator:Found 63 codons ATT
INFO:u

Fetching seqs:   0%|          | 0/10472 [00:00<?, ?it/s]

,Seq,IsPositive
0,AGCTCGGAAGACCGCGAGGCGCGCTTTTCTGACGCATCGGCGCCTT...,1
1,GCGTCCTCCTGCCGGCCTGCAGGCCCGGGGCCTCCGCCTGCTTCCC...,1
2,GGCTAATTCCGAATTCCAAATCGGAAGCAAGAGGGCGGGGCCCCGT...,1
3,GCTCGTGGACTAGCTCCGCCTCCTAGCGTCTGACTACCCCCTCAGC...,1
4,CTGCAGGTACGCGCGGGCCGGGCGGGGCGGGCGGGCGGCGGGCGCG...,1


In [8]:
# Turn on the tokenization of the DNA sequence into k-mers
dsg.kmer_size = 6
dsg().head()

INFO:uBERTa.datasets_generator:Obtained a dataset with 830191 records
INFO:uBERTa.datasets_generator:Found 7426 positive and 822765 negative records
INFO:uBERTa.datasets_generator:Will sample 3680=4600 * 0.8 positive examples of group u
INFO:uBERTa.datasets_generator:Will sample 388=485 * 0.8 positive examples of group ma
INFO:uBERTa.datasets_generator:Will sample 1170=2341 * 0.5 positive examples of group m
INFO:uBERTa.datasets_generator:Sampled 5238 positive examples
INFO:uBERTa.datasets_generator:Expecting around 5238 total negative examples
INFO:uBERTa.datasets_generator:Num samples after the per-codon correction: 5236
INFO:uBERTa.datasets_generator:Found 1141 codons ATG
INFO:uBERTa.datasets_generator:Found 752 codons CTG
INFO:uBERTa.datasets_generator:Found 289 codons GTG
INFO:uBERTa.datasets_generator:Found 140 codons TTG
INFO:uBERTa.datasets_generator:Found 137 codons ACG
INFO:uBERTa.datasets_generator:Found 71 codons ATC
INFO:uBERTa.datasets_generator:Found 68 codons ATT
INFO:u

Fetching seqs:   0%|          | 0/10474 [00:00<?, ?it/s]

,Seq,IsPositive
0,GAGCGT AGCGTT GCGTTG CGTTGC GTTGCG TTGCGG TGCG...,1
1,CCGGGG CGGGGC GGGGCT GGGCTA GGCTAC GCTACA CTAC...,1
2,AGTGTT GTGTTT TGTTTG GTTTGT TTTGTT TTGTTA TGTT...,1
3,ACTCCC CTCCCA TCCCAC CCCACC CCACCG CACCGA ACCG...,1
4,GGCTGA GCTGAA CTGAAT TGAATC GAATCG AATCGC ATCG...,1
